In [1]:
import pandas as pd,numpy as np,requests,re
from bs4 import BeautifulSoup
import textstat

In [2]:
ofmiceandmen2 = text_file = open("OfMiceAndMen2.txt", "r")
ofmiceandmen2 = ofmiceandmen2.read().split('. ')
ofmiceandmen2 = pd.DataFrame(ofmiceandmen2)

In [3]:
ofmiceandmen = text_file = open("ofmiceandmen.txt", "r")
ofmiceandmen = ofmiceandmen.read().split('. ')
ofmiceandmen = pd.DataFrame(ofmiceandmen)

In [4]:
mrtickle = text_file = open("mrtickle.txt", "r")
mrtickle = mrtickle.read().split('. ')
mrtickle = pd.DataFrame(mrtickle)

In [5]:
mrgreedy = text_file = open("mrgreedy.txt", "r")
mrgreedy = mrgreedy.read().split('. ')
mrgreedy = pd.DataFrame(mrgreedy)

In [6]:
canneryrow = text_file = open("Canneryrow.txt", "r")
canneryrow = canneryrow.read().split('. ')
canneryrow = pd.DataFrame(canneryrow)

In [7]:
#Merge mr men and of mice and men

books = pd.DataFrame({
                    "Title": [ofmiceandmen[0][0],ofmiceandmen2[0][0],mrtickle[0][0],mrgreedy[0][0],canneryrow[0][0]],
                    "Text": [ofmiceandmen[0][1:].str.cat(sep = ". "),ofmiceandmen2[0][1:].str.cat(sep = ". "),mrtickle[0][1:].str.cat(sep = ". "),mrgreedy[0][1:].str.cat(sep = ". "),canneryrow[0][1:].str.cat(sep = ". ")]
                    })

In [8]:
#Creates list of book url links to retrieve text in next loop

UrlList = []
BookIndex = 1500
BookSearchRange = 50

for i in range(BookSearchRange):
    
    UrlList.append("https://www.gutenberg.org/files/" + str(int(BookIndex+len(UrlList))) + "/" +
                   str(int(BookIndex + len(UrlList))) + "-0.txt")   

print(UrlList[-1])

https://www.gutenberg.org/files/1549/1549-0.txt


In [9]:
#loop to retrieve book text and append to main Data Frame

GutenbergBooks = pd.DataFrame()

BookTitleFinder = re.compile('(?<=Title: ).{,}')

for i in UrlList:
    
    r=requests.get(i)
    soup = BeautifulSoup(r.text,"html.parser")
    if len(soup.text) > 100:
        temp = pd.DataFrame({
                             "Title":[BookTitleFinder.findall(soup.text)]
                            ,"Text":[soup]
                            })
        
        GutenbergBooks = GutenbergBooks.append(temp).reset_index(drop=True)

In [10]:
#Clean the titles as they have []

for i, j in enumerate(GutenbergBooks.Title):
    GutenbergBooks.loc[i,"Title"] = re.sub("\[|'|]|r]|\\\\", "", str(j))

In [11]:
#Define function to find nth index position of substring in string

def findnth(string, substring, n):
    parts = string.split(substring, n + 1)
    if len(parts) <= n + 1:
        return -1
    return len(string) - len(parts[-1]) - len(substring)

In [12]:
#Create text sample column and drop text

books["TextSample"] = 0
for i, j in enumerate(GutenbergBooks.Text):
    Example = str(GutenbergBooks.Text[i])
    Start = findnth(Example, ".", 50)+2
    End = findnth(Example, ".", 350)
    GutenbergBooks.loc[i,"TextSample"] = str(GutenbergBooks.Text[i])[Start:End]
    temp = re.sub("\r","",str(GutenbergBooks.TextSample[i]))
    GutenbergBooks.loc[i,"TextSample"] = re.sub("\n"," ",temp)
    #GutenbergBooks.loc[i,"TextSample"] = re.sub("\A\S+\s+","",str(GutenbergBooks.TextSample[i]))

In [13]:
#Remove dirty text

for i, j in enumerate(GutenbergBooks.TextSample):
    GutenbergBooks.loc[i,"TextSample"] = re.sub("\s{1,}"," ",str(j))
    GutenbergBooks.loc[i,"TextSample"] = re.sub("\.\s{1,}\."," ",str(j))
    GutenbergBooks.loc[i,"TextSample"] = re.sub("-{2,}"," ",str(j))
    GutenbergBooks.loc[i,"TextSample"] = re.sub("\s"," ",str(j))

In [14]:
#Merge gutenberg books and books

books = books[["Title","Text"]]
GutenbergBooks = GutenbergBooks[["Title","TextSample"]]
GutenbergBooks = GutenbergBooks.rename(index=str, columns={"TextSample": "Text"})

books = books.append(GutenbergBooks)
books = books.reset_index()

In [15]:
#Create dummy columns

books['SMOG'] = 0
books['FleschGrade'] = 0
books['AutomatedReadability'] = 0
books['GunningFOG'] = 0

In [16]:
#Add readability metrics

for i, j in enumerate(books.Text):
    books.loc[i,"SMOG"] = textstat.smog_index(str(books.Text[i]))
    books.loc[i,"FleschGrade"] = textstat.flesch_kincaid_grade(str(books.Text[i]))
    books.loc[i,"AutomatedReadability"] = textstat.automated_readability_index(str(books.Text[i]))
    books.loc[i,"GunningFOG"] = textstat.gunning_fog(str(books.Text[i]))

In [17]:
books.head(10)

,index,Title,Text,SMOG,FleschGrade,AutomatedReadability,GunningFOG
0,0,Of Mice & Men,"A few miles south of Soledad, the Salinas Rive...",6.7,5.4,7.3,7.50
1,1,Of Mice & Men Sample,"A few miles south of Soledad, the Salinas Rive...",7.8,13.5,18.0,14.87
2,2,Mr Tickle,"It was a warm, sunny morning. In his small hou...",6.6,4.9,6.1,5.69
3,3,Mr Greedy Sample,"Mr Greedy liked to eat! In fact, Mr Greedy lov...",3.1,2.4,1.1,3.88
4,4,Cannery Row,"Lee Chong’s grocery, wliile not a model of nea...",8.8,7.6,9.5,9.25
5,0,The Taming of the Shrew,You will not pay for the glasses you have burs...,8.0,7.2,8.9,8.49
6,1,Romeo and Julietr,Mantua. A Street. Scene II. Friar Lawrence’s C...,7.9,5.2,6.8,6.58
7,2,A Midsummer Night’s Dreamr,HERMIA. So is Lysander. THESEUS. In himself...,7.8,6.5,8.4,7.88
8,3,The Merchant of Venicer,Belmont. A room in Portia’s house. Scene V. Th...,8.7,6.8,8.3,8.38
9,4,Much Ado about Nothing,"FRIAR FRANCIS. A Sexton. A Boy. HERO, Daughte...",7.5,6.4,7.3,7.58


In [18]:
books.to_csv("Output.csv")